In [1]:
import tensorflow as tf
import numpy as np
import os
import time

# 세익스피어 파일 다운로드
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 0us/step


In [2]:
# 읽은 다음 파이썬 2와 호환되도록 디코딩합니다.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트의 길이는 그 안에 있는 문자의 수입니다.
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# 파일의 고유 문자수를 출력합니다.
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 65개


In [5]:
# 고유 문자에서 인덱스로 매핑 생성
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [6]:
# 텍스트에서 처음 13개의 문자가 숫자로 어떻게 매핑되었는지를 보여줍니다
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
# 단일 입력에 대해 원하는 문장의 최대 길이
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 훈련 샘플/타깃 만들기 / 원하는 만큼 잘라낼 수 있는 함수
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [9]:
# batch 메서드는 이 개별 문자들을 원하는 크기의 시퀀스로 쉽게 변환할 수 있습니다.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [10]:
# train / test split
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(2):
  print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
입력 데이터:  'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
타깃 데이터:  're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [13]:
# 이 벡터의 각 인덱스는 하나의 타임 스텝(time step)으로 처리됩니다. 
# 타임 스텝 0의 입력으로 모델은 "F"의 인덱스를 받고 다음 문자로 "i"의 인덱스를 예측합니다. 
# 다음 타임 스텝에서도 같은 일을 하지만 RNN은 현재 입력 문자 외에 이전 타임 스텝의 컨텍스트(context)를 고려합니다.

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 39 ('a')
  예상 출력: 56 ('r')
   1단계
  입력: 56 ('r')
  예상 출력: 43 ('e')
   2단계
  입력: 43 ('e')
  예상 출력: 1 (' ')
   3단계
  입력: 1 (' ')
  예상 출력: 39 ('a')
   4단계
  입력: 39 ('a')
  예상 출력: 50 ('l')


In [18]:
# 텍스트를 다루기 쉬운 시퀀스로 분리하기 위해 tf.data를 사용했습니다. 
# 그러나 이 데이터를 모델에 넣기 전에 데이터를 섞은 후 배치를 만들어야 합니다.

# 배치 크기
BATCH_SIZE = 64

# 데이터셋을 섞을 버퍼 크기
# (TF 데이터는 무한한 시퀀스와 함께 작동이 가능하도록 설계되었으며,
# 따라서 전체 시퀀스를 메모리에 섞지 않습니다. 대신에,
# 요소를 섞는 버퍼를 유지합니다).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 모델 설계

모델을 정의하려면 tf.keras.Sequential을 사용합니다. 이 간단한 예제에서는 3개의 층을 사용하여 모델을 정의합니다:

- tf.keras.layers.Embedding : 입력층. embedding_dim 차원 벡터에 각 문자의 정수 코드를 매핑하는 훈련 가능한 검색 테이블.
- tf.keras.layers.GRU : 크기가 units = rnn_units인 RNN의 유형(여기서 LSTM층을 사용할 수도 있습니다.
- tf.keras.layers.Dense : 크기가 vocab_size인 출력을 생성하는 출력층.

In [19]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)

# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 
                              embedding_dim,
                              batch_input_shape = [batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences = True,
                        stateful = True,
                        recurrent_initializer = 'glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [21]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [31]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
#   print(example_batch_predictions)
  print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

tf.Tensor(
[[[ 5.5931852e-04 -4.1044932e-03 -1.7231426e-03 ...  3.2231417e-03
    1.1348339e-02  9.3371999e-03]
  [ 1.5804544e-03 -2.4458161e-03 -1.6484198e-03 ...  1.4858423e-03
    1.6409174e-02  1.1202208e-02]
  [-7.4341538e-04 -4.1244724e-03 -3.0674415e-03 ...  3.2579007e-03
    1.3385746e-02  9.0167690e-03]
  ...
  [-7.7964356e-03 -2.6391575e-03 -7.8454362e-03 ...  8.6353617e-03
    4.4174474e-03  5.4403255e-03]
  [-1.0220366e-02 -4.9307453e-03 -6.0157310e-03 ...  5.9397547e-03
    2.1903145e-03  8.9474460e-03]
  [-1.0970656e-02 -5.1122829e-03 -6.9101062e-03 ...  1.0959120e-02
    2.9901166e-03  9.2035048e-03]]

 [[ 1.3965892e-03 -2.2105637e-03 -2.0963652e-03 ...  5.7412898e-03
    3.5536485e-03  2.1745190e-03]
  [-4.1844189e-04 -7.7693682e-04 -6.5573701e-03 ...  7.8514395e-03
    6.8277018e-03 -4.5149075e-03]
  [ 4.1518128e-05 -2.3287901e-04 -5.1847780e-03 ...  9.7866682e-03
    5.4392247e-03 -9.0550649e-04]
  ...
  [-9.6385088e-03  9.0300362e-04  6.1996151e-03 ...  7.3643995e-04

In [29]:
print(dataset.take(1))


<TakeDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print(sampled_indices)

[17 53 41 21 52 45  8 56 19 45 40 37 29 50 59 41 21 35 56 22 50  9 62 36
 33 50 61 55 50  5 57 28 42 52 43 13 11 57 12 48 39  2 24 44 19 31 24 23
 15 12  5 49 23 55 49 18  8 26 22 55 31 48 34 21 20 51 58 16 58  7 16  3
 63 47 33 61 12 63 55 53 20 19 44 31 22 61 43  7 19 49 54 59  5 61 13  0
 41 13 18 25]


In [28]:
# pretrained model prediction result

print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 '.\nWe recommend to you, tribunes of the people,\nOur purpose to them: and to our noble consul\nWish we '

예측된 다음 문자: 
 "EocIng.rGgbYQlucIWrJl3xXUlwql'sPdneA;s?ja!LfGSLKC?'kKqkF.NJqSjVIHmtDt-D$yiUw?yqoHGfSJwe-Gkpu'wA\ncAFM"


In [32]:
# 손실함수 정의
# 출력값이 로짓의 형태이기 때문에 from_logit = True

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.1736355


In [33]:
# tf.keras.Model.compile 메서드를 사용하여 훈련 절차를 설정합니다. 
# 기본 매개변수의 tf.keras.optimizers.Adam과 손실 함수를 사용합니다.

model.compile(optimizer='adam', loss=loss)

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [35]:
#체크포인트 구성
# tf.keras.callbacks.ModelCheckpoint를 사용하여 훈련 중 체크포인트(checkpoint)가 저장되도록 합니다.

# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
EPOCHS = 3

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
     13/Unknown - 116s 9s/step - loss: 3.5216

KeyboardInterrupt: 